In [ ]:
import numpy as np
import math as mt
import random as rd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def threedplot(crd) :
  crd_x = np.array([crd[i][0] for i in range(len(crd))])
  crd_y = np.array([crd[i][1] for i in range(len(crd))])
  crd_z = np.array([crd[i][2] for i in range(len(crd))])

  plt.rcParams["figure.figsize"] = (6, 6)
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  ax.scatter(crd_x, crd_y, crd_z, marker='o', s=15, cmap='Greens')
  plt.show()


class MC : 
  def __init__(self, npart, density, temp, maxdr, nstep, printfreq):
    self.npart = npart
    self.dens = density  
    self.temp = temp
    self.maxdr = maxdr 
    self.nstep = nstep
    self.print = printfreq
    self.V = self.npart/self.dens
    self.beta = 1.0/self.temp
    coords = np.zeros((self.npart, 3))
    L = (self.npart/self.dens)**(1.0/3)
    self.samplefreq = 10
    self.samplecount= 0 
    self.avgL = 0
   # self.coords = coords
    self.L = L
    print(self.L, 'L')
    for part in range(0, self.npart):
      index = [(rd.randint(0, 1000)/1000.0)-0.5, (rd.randint(0, 1000)/1000.0)-0.5 , (rd.randint(0, 1000)/1000.0)-0.5]
      coords[part,:] = np.dot(index,self.L) ### all same initial coord
    self.coords = coords
   # return coords, L 
  #  print(coords)



  def pbc3d(self, vec, L) : 
    for dim in range (0,3):
      if vec[dim] > L/2 : 
        vec[dim] = vec[dim] - L 
      elif vec[dim] < -L/2 : 
        vec[dim] = vec[dim] + L
    return vec

  def distpbc3d(self, vector, a) : 
    ha = a / 2.0
    for dim in range (0,3):
      if vector[dim] > ha: 
        vector[dim] = vector[dim] - a
      elif vector[dim] < -ha:
        vector[dim] = vector[dim] + a
    return vector

  def distpbc3dold(self, old, a) : 
    ha = a / 2.0
    for dim in range (0,3):
      if old[dim] > ha: 
        old[dim] = old[dim] - a
      elif old[dim] < -ha:
        old[dim] = old[dim] + a
    return old
  
  def LJ_E(self, mat,L) : 
    self.energy = 0
    virial = 0
    for partA in range (0,self.npart-1):
      for partB in range (partA + 1, self.npart):
        dr = mat[partA,:] - mat[partB,:]
        dr = self.distpbc3d(dr, L)
        dr2 = dr[0]**2 + dr[1]**2 + dr[2]**2
        invdr6 = 1.0 /(dr2**3) ###/r^6
        self.energy += (invdr6 * ( invdr6 -1))
        virial +=  invdr6**2
    energy = self.energy * 4
    vir = (self.energy + virial)*24.0/3.0
    self.dens = self.npart/L**3
    pressure = (self.dens/self.beta) + (vir/L**3)
    density = self.dens
    return energy, vir, pressure, density



        
  def LJ_E_change(self, coords, trialpos, i, L) : 
    deltaE = 0 
    self.i = i
    
    for otherpart in range (0, self.npart):
      if otherpart == self.i :
        continue
      drnew = coords[otherpart,:] - trialpos
      drold = coords[otherpart,:] - coords[self.i,:]

      # pbc
      drnew_pbc = self.distpbc3d(drnew, L)
      drold_pbc = self.distpbc3dold(drold, L)

      # get the dist sqared
      drnew2 = (drnew_pbc[0]**2 + drnew_pbc[1]**2 + drnew_pbc[2]**2)
      drold2 = (drold_pbc[0]**2 + drold_pbc[1]**2 + drold_pbc[2]**2)  


  #    print(drold2,'drold2')
      invdrnew6 = 1.0 / (drnew2**3)
      invdrold6 = 1.0 / (drold2**3)
      
      enew = (invdrnew6 * (invdrnew6 - 1))
      eold = (invdrold6 * (invdrold6 - 1))
      
      deltaE = deltaE + enew - eold
    deltaE = deltaE * 4
    return deltaE
   

  def nvt(self): 
 #   self.config()
    #threedplot(self.coords)
    E, vir, P , rho = self.LJ_E(self.coords, self.L)
    print(E)
    rand = np.random.rand(1,3)
    for step in range (0, self.nstep): 
      if step % self.print ==0: 
        step
      for i in range (0, self.npart):
        rand = np.array([(rd.randint(0, 1000)/1000.0), (rd.randint(0, 1000)/1000.0) , (rd.randint(0, 1000)/1000.0)])
        rtrial = self.coords[i,:] + np.array(np.dot(self.maxdr,(rand-0.5)))
        rtrial_pbc = self.pbc3d(rtrial, self.L)
        deltaE = self.LJ_E_change(self.coords, rtrial_pbc, i, self.L)
      
        if np.all(rand < np.exp(-self.beta*deltaE)):
          self.coords[i,:] = rtrial_pbc
          E += deltaE


    coordinates = self.coords
    #threedplot(coordinates)
    finalE, finalvir, finalP, finalR = self.LJ_E(coordinates, self.L)
    return coordinates, E, self.dens, finalE, finalvir, finalP, finalR
        ####변화 plot

 
  def npt(self, maxdv, press): 
    self.maxdv = maxdv
    self.press = press
    #threedplot(self.coords)
    E, vir, P , rho= self.LJ_E(self.coords, self.L)
    print('presstrial', P)
    for step in range (0, self.nstep): 
        rand = np.random.rand(1,3)
        if np.all(rand*(self.npart+1)+1 < self.npart):

            for i in range (0, self.npart):

                rand = np.array([(rd.randint(0, 1000)/1000.0), (rd.randint(0, 1000)/1000.0) , (rd.randint(0, 1000)/1000.0)])
                rtrial = self.coords[i,:] + np.array(np.dot(self.maxdr,(rand-0.5)))
                rtrial_pbc = self.pbc3d(rtrial, self.L)
                deltaE = self.LJ_E_change(self.coords, rtrial_pbc, i, self.L)
      
                if np.all(rand < np.exp(-self.beta*deltaE)):
                    self.coords[i,:] = rtrial_pbc
                    E += deltaE

        else :
 # perform volume change move
            oldv = self.L**3
            lnvtrial = np.log(oldv) + (np.random.rand() - 0.5)*self.maxdv
        
            vtrial = np.exp(lnvtrial)
            newL = vtrial**(1.0/3)
            coordstrial = self.coords*(newL/self.L)
            etrial, virtrial, Ptrial, rhotrial = self.LJ_E(coordstrial, newL)
            weight = (etrial - E) + self.press*(vtrial - oldv) - ((self.npart+1)/self.beta)*np.log(vtrial/oldv)
            if np.random.rand() < np.exp(-self.beta*weight):
                self.coords = coordstrial
                E = etrial
                self.L = newL
                print('ptrial', Ptrial, 'l', self.L)
          #listL.append(self.L)
          

        if step % self.samplefreq == 0 :
            self.samplecount += 1
            self.avgL += self.L

      
    
      


    coordinates = self.coords
    avgL = self.avgL / self.samplecount
    #threedplot(coordinates)
    finalE, finalvir, finalP, finalR = self.LJ_E(coordinates, self.L)
    return coordinates, E, avgL, finalE, finalvir, finalP, finalR

  def npt_v(self,maxdv, press):
    self.maxdv = maxdv
    self.press = press 
   # threedplot(self.coords)
    E, vir, press, rho = self.LJ_E(self.coords, self.L)
    print('init', press)
    for step in range (0, self.nstep): 
        rand = np.random.rand(1,3)
        if np.all(rand*(self.npart+1)+1 < self.npart):

            for i in range (0, self.npart):

                rand = np.array([(rd.randint(0, 1000)/1000.0), (rd.randint(0, 1000)/1000.0) , (rd.randint(0, 1000)/1000.0)])
                rtrial = self.coords[i,:] + np.array(np.dot(self.maxdr,(rand-0.5)))
                rtrial_pbc = self.pbc3d(rtrial, self.L)
                deltaE = self.LJ_E_change(self.coords, rtrial_pbc, i, self.L)
      
            if np.all(rand < np.exp(-self.beta*deltaE)):
                self.coords[i,:] = rtrial_pbc
                E += deltaE

        else :
 # perform volume change move
            oldv = self.L**3
            vtrial = oldv + (np.random.rand() - 0.5)*self.maxdv
        
        #vtrial = np.exp(lnvtrial)
            newL = vtrial**(1.0/3)
            coordstrial = self.coords*(newL/self.L)
            etrial, virtrial, presstrial, rhotrial = self.LJ_E(coordstrial, newL)
            print('presstrial', presstrial)
            weight = (etrial - E) + self.press*(vtrial - oldv) - (self.npart+1)*self.temp*np.log(vtrial/oldv)
        
            if np.random.rand() < np.exp(-self.beta*weight):
                self.coords = coordstrial
                E = etrial
                self.L = newL
        
          

        if step % self.samplefreq == 0 :
            self.samplecount += 1
            self.avgL += self.L

      

      


    coordinates = self.coords
    avgL = self.avgL / self.samplecount
   # threedplot(coordinates)
    finalE, finalvir, finalP, finalR = self.LJ_E(coordinates, self.L)
    return coordinates, E, avgL, finalE, finalvir, finalP, finalR



